In [1]:
import orjson
import glob
import os
import pandas as pd

# jsonl 파일들이 있는 폴더 경로
base_path = "G:/내 드라이브/telemetry_for_top15/"

# 파일명 조건 설정 (여러 조건 추가 가능)
file_patterns = [
    "*squad_Baltic_Main.jsonl", "*squad_Tiger_Main.jsonl", "*squad_Desert_Main.jsonl"
]

# 모든 조건에 맞는 파일 수집
files = []
for pattern in file_patterns:
    matched_files = glob.glob(os.path.join(base_path, pattern))
    files.extend(matched_files)

# 중복 제거
files = list(set(files))

high_ai_matches = []

print(f"총 {len(files)}개의 파일을 분석합니다...")

for file in files:
    try:
        with open(file, 'rb') as f:
            for line in f:
                try:
                    data = orjson.loads(line)
                    # LogMatchStart 이벤트 찾기 (플레이어 정보가 여기에 포함됨)
                    if data.get('_T') == 'LogMatchStart':
                        characters = data.get('characters', [])
                        
                        total_players = len(characters)
                        if total_players == 0:
                            continue

                        ai_count = 0
                        for char_info in characters:
                            # 캐릭터 정보 구조에 따라 접근 (characters 리스트 안에 character 객체가 있음)
                            c = char_info.get('character', {})
                            if c.get('type') == 'user_ai':
                                ai_count += 1
                        
                        ratio = ai_count / total_players
                        
                        # AI 비율이 0.4 이상인 경우 리스트에 추가
                        if ratio >= 0.4:
                            high_ai_matches.append({
                                'filename': os.path.basename(file),
                                'filepath': file,
                                'total_players': total_players,
                                'ai_count': ai_count,
                                'ai_ratio': ratio
                            })
                            print(f"파일: {os.path.basename(file)} | 총원: {total_players}명 | AI: {ai_count}명 ({ratio*100:.1f}%)")
                        break # 한 파일에서 LogMatchStart를 찾으면 다음 파일로 넘어감
                except:
                    continue
    except Exception as e:
        print(f"Error reading {file}: {e}")

# 결과를 DataFrame으로 변환하고 CSV로 저장
if high_ai_matches:
    df = pd.DataFrame(high_ai_matches)
    output_path = 'high_ai_matches.csv'
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n[분석 결과 요약]")
    print(f"전체 파일 수: {len(files)}")
    print(f"AI 비율 40% 이상 매치 수: {len(high_ai_matches)}")
    print(f"평균 AI 비율: {df['ai_ratio'].mean()*100:.2f}%")
    print(f"\nCSV 파일이 '{output_path}'로 저장되었습니다.")
else:
    print("\nAI 비율이 40% 이상인 매치를 찾지 못했습니다.")


총 3848개의 파일을 분석합니다...
파일: 20251107_155815_squad_Baltic_Main.jsonl | 총원: 97명 | AI: 44명 (45.4%)
파일: 20251112_183432_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 91명 (91.0%)
파일: 20251115_032551_squad_Tiger_Main.jsonl | 총원: 100명 | AI: 86명 (86.0%)
파일: 20251106_165331_squad_Tiger_Main.jsonl | 총원: 89명 | AI: 40명 (44.9%)
파일: 20251110_075237_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 91명 (91.0%)
파일: 20251115_095317_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 91명 (91.0%)
파일: 20251110_115421_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 91명 (91.0%)
파일: 20251111_112005_squad_Baltic_Main.jsonl | 총원: 95명 | AI: 44명 (46.3%)
파일: 20251115_144225_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 92명 (92.0%)
파일: 20251114_134508_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 90명 (90.0%)
파일: 20251107_154725_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 92명 (92.0%)
파일: 20251106_165331_squad_Tiger_Main.jsonl | 총원: 89명 | AI: 40명 (44.9%)
파일: 20251110_075237_squad_Baltic_Main.jsonl | 총원: 100명 | AI: 91명 (91.0%)
파일: 20251115_095317_squad_Baltic_Mai